In [1]:
import telebot
import transformers
from transformers import pipeline
import fitz

In [3]:
# Abre o documento PDF
doc = fitz.open('calendario_universitario.pdf')

# Itera sobre cada página
context = ''
for page in doc:
    # Extrai o texto por blocos, preservando melhor a formatação
    blocks = page.get_text("blocks")

    for block in blocks:
        if block[0] != 0:  # Filter out non-text blocks
            context += block[4]

model_name = 'pierreguillou/bert-base-cased-squad-v1.1-portuguese'
nlp = pipeline("question-answering", model=model_name)

question = "Quando será o cadastro de novos alunos em mobilidade nacional?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: '22 de abril a 21 de junho
de 2024', score: 0.2084, start: 409, end: 442


In [4]:
bot = telebot.TeleBot(KEY_API)

In [5]:
def check(message):
    return True

In [6]:
@bot.message_handler(commands=['start'])
def say_hi(message):
    bot.send_message(message.chat.id, f"Olá, {message.from_user.first_name}")

In [7]:
@bot.message_handler(func=check)
def respond(message):
    result = nlp(question=message.text, context=context)
    bot.reply_to(text=result['answer'], message=message)

In [ ]:
bot.polling()